# Testing Tuned Models

In [1]:
%matplotlib inline
from IPython.core.display import set_matplotlib_formats
import matplotlib
import matplotlib.pyplot as plt
import os.path
import shutil
import pandas as pd
import numpy as np
import os
import json
import gc
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn import metrics
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback
import tensorflow as tf
from common.model_utils import build_model, input_size
from tensorflow.keras.applications import efficientnet as efn
from tensorflow.keras.models import load_model
import time

# from common.landmark_model_utils import build_model_landmarks

# import packages and modules for graph plotting
from pandas.plotting import table

# setup output image format (Chrome works best)
set_matplotlib_formats("svg")

/tmp/ipykernel_27735/1029250346.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
2024-04-22 23:26:31.941331: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/tmp/ipykernel_27735/1029250346.py:29: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats("svg")


In [2]:
data_folder = "/home/FYP/dion0020/deepfake-detector/dev/Training Data"
landmark_data_folder = "/home/FYP/dion0020/deepfake-detector/dev/Training Landmarks Data"
result_folder = "/home/FYP/dion0020/deepfake-detector/dev/Result"
landmark_result_folder = "/home/FYP/dion0020/deepfake-detector/dev/Landmark Result"
models_folder = "/home/FYP/dion0020/deepfake-detector/dev/Benchmark Models"
landmark_models_folder = "/home/FYP/dion0020/deepfake-detector/dev/Landmark Models"
# landmark_models_folder = "/home/FYP/dion0020/deepfake-detector/dev/Landmark Models/old"
tuning_folder = "/home/FYP/dion0020/deepfake-detector/dev/Tuning"
landmark_tuning_folder = "/home/FYP/dion0020/deepfake-detector/dev/Landmark Tuning"

BATCH_SIZE = 16

LANDMARK = True
model_type = "EfficientNetB0"
models_result = {}

In [3]:
def read_hyperparameters_file(file_path):
    # Initialize an empty dictionary to store hyperparameters
    hyperparameters = {}

    try:
        # Open the file and read its contents
        with open(file_path, 'r') as file:
            # Read each line in the file
            for line in file:
                # Split each line into key-value pairs
                key, value = line.strip().split(': ')
                # Store the key-value pair in the dictionary
                hyperparameters[key] = value
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
    except Exception as e:
        print(f"Error: {e}")

    return hyperparameters

In [4]:

if LANDMARK:
    tuning_dir = landmark_tuning_folder
    data_dir = landmark_data_folder
    model_dir = landmark_models_folder
    result_dir = landmark_result_folder
else:
    tuning_dir = tuning_folder
    data_dir = data_folder
    model_dir = models_folder
    result_dir = result_folder

In [5]:
def copy_misclassified_images(actual_labels, predicted_labels, filenames, src_dir, dest_dir):
    for i in range(len(actual_labels)):
        if actual_labels[i] != predicted_labels[i]:
            if filenames[i].split('_')[0] == "faked":
                file = "manipulated"
            else:
                file = "original"
            shutil.copy(os.path.join(src_dir, f"{file}/{filenames[i]}"), os.path.join(dest_dir, filenames[i]))

In [6]:
# file_path = f"{tuning_dir}/{model_type} Tuning/{model_type}_best_hyperparameters.txt"
model_path = os.path.join(".", model_dir, f"{model_type}" + ".hdf5")
# model_path = "/home/FYP/dion0020/deepfake-detector/dev/Landmark Models/old/EfficientNetB0.hdf5"
# best_hp = read_hyperparameters_file(file_path)

# Specify the path to the test data
test_path = os.path.join(".", data_dir, "Test")

total_prediction_time = 0

misclassified_images_dir = os.path.join(".", data_dir, "misclassified_images")
if not os.path.exists(misclassified_images_dir):
    os.makedirs(misclassified_images_dir)

# Test the models
print("Starting prediction for", model_type)
test_img_gen = ImageDataGenerator(
    preprocessing_function=efn.preprocess_input
)

imgsize = input_size(model_type)

test_generator = test_img_gen.flow_from_directory(
    directory=test_path,
    target_size=(imgsize, imgsize),
    batch_size=BATCH_SIZE,
    class_mode=None,
    shuffle=False,
    seed=123,
)

model = load_model(model_path)

print("Loaded model {} from disk".format(model_type))

start_time = time.time()
predicted = model.predict(test_generator, steps=len(test_generator), verbose=1)
end_time = time.time()

inference_time = (end_time - start_time) / len(test_generator)
predicted_class_indices = np.argmax(predicted, axis=1)
y = test_generator.classes

Accuracy = metrics.accuracy_score(y, predicted_class_indices)
print("Model Accuracy: ", Accuracy)

Precision = metrics.precision_score(y, predicted_class_indices, average="binary")
print("Model Precision: ", Precision)

Recall = metrics.recall_score(y, predicted_class_indices, average="binary")
print("Model Recall: ", Recall)

Auc = metrics.roc_auc_score(y, predicted_class_indices)
print("AUC: ", Auc, "\n")
print("Inference Time: ", inference_time)

models_result[model_type] = {
    "Accuracy": Accuracy,
    "Precision": Precision,
    "Recall": Recall,
    "AUC": Auc,
}

filenames = [os.path.split(i)[1] for i in test_generator.filenames]
actualLabel = [os.path.split(i)[0] for i in test_generator.filenames]

for i in range(len(actualLabel)):
    if actualLabel[i] == "manipulated":
        actualLabel[i] = "0"
    else:
        actualLabel[i] = "1"

results_df = pd.DataFrame(
    {
        "id": pd.Series(filenames),
        "actual label": pd.Series(actualLabel),
        "pred label": pd.Series(predicted_class_indices),
    }
)
os.makedirs(result_dir, exist_ok=True)
results_df.to_csv(os.path.join(".", result_dir, model_type + "_out.csv"))

with open(os.path.join(".", result_dir, "model_results.json"), "w") as outfile:
    json.dump(models_result, outfile)

model_prediction_results = pd.DataFrame.from_dict(models_result, orient="index")
model_prediction_results.to_csv(
os.path.join(".", result_dir, "prediction_results.csv")
)

Error: File '/home/FYP/dion0020/deepfake-detector/dev/Landmark Tuning/EfficientNetB0 Tuning/EfficientNetB0_best_hyperparameters.txt' not found.
Starting prediction for EfficientNetB0
Found 42144 images belonging to 2 classes.


2024-04-22 23:26:39.132716: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-22 23:26:39.132747: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-04-22 23:26:39.132780: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SCSEGPU-TC1): /proc/driver/nvidia/version does not exist
2024-04-22 23:26:39.133339: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


TypeError: ('Keyword argument not understood:', 'invert')